## Requirements

In [ ]:
#!pip install elasticsearch==7.13
#!pip install configparser

In [59]:
from configparser import ConfigParser
from elasticsearch import Elasticsearch
import re

from random import randint

### Secrets

In [8]:
parser = ConfigParser()
_ = parser.read('../notebook.cfg')
kibana_url = parser.get('bonsai', 'kibana_url')

### ElasticSearch Create
Docs: https://elasticsearch-py.readthedocs.io/en/v7.13.0/

In [16]:
auth = re.search('https\:\/\/(.*)\@', kibana_url).group(1).split(':')
host = kibana_url.replace('https://%s:%s@' % (auth[0], auth[1]), '')

# optional port
match = re.search('(:\d+)', host)
if match:
    p = match.group(0)
    host = host.replace(p, '')
    port = int(p.split(':')[1])
else:
    port = 443

# Connect to cluster over SSL using auth for best security:
es_header = [{
    'host': host,
    'port': port,
    'use_ssl': True,
    'http_auth': (auth[0],auth[1])
}]

In [17]:
es = Elasticsearch(es_header)

In [18]:
es.ping()

True

In [20]:
es.indices.refresh(index="hello_world")

{'_shards': {'total': 2, 'successful': 2, 'failed': 0}}

In [70]:
getBody = {
    "query": {
        "match_all": {}
    }
}

res = es.search(index="hello_world", body=getBody)
print("Got %d Hits:" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print(hit["_source"])

Got 2 Hits:
{'name': 'Matheus Ervilha', 'age': '29'}
{'name': 'Matheus Ervilha 840', 'age': '30'}


In [69]:
newDoc = {
    'name': 'Matheus Ervilha ' + str(randint(0, 10000)),
    'age': str(randint(0, 100)),
}
res = es.index(index="hello_world", id=1, body=newDoc)
print(res['result'])

created
